In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Referred by - share row# I name I city (this field is not mandatory - those who are referred need to fill this),Video Introduction
0,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Health concious due to health conditions in fa...,I give consent,,,,,,,,
1,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,I like playing sports,I give consent,,,,,,,,
2,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,No,I give consent,,,,,,,,
3,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,I take time to open up and quite curious,I give consent,,,,,,,,
4,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,I believe honest communication and mutual resp...,I give consent,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,Male,36,Pune,,"A life with a few friends bt real ones, a job ...",Grew up in a middle class strict family to rea...,"Was married earlier, legally divorced",No,Between 18-20 Lakhs,I am a professional Chef,...,"I love cars, bikes and dogs \nWould help an an...",I give consent,Hindu,Yes,I am not sure,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
435,Male,38,Delhi,,"Healthy, loving family and good job",Awesome,"Was married earlier, legally divorced",No,Between 40-50 Lakhs,IT professional,...,No,I give consent,Hindu,Yes,It will not work for me,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
436,Female,31,Bangalore,,Having someone cheer for you when you follow y...,Growing up years were decent. No major complai...,Never married,No,Between 40-50 Lakhs,Software engineer,...,"Into Fitness, beaches and baby elephants :P Re...",I give consent,Other,I am not sure,It will not work for me,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
437,Male,33,Bangalore,,"Supportive, Caring, Patience.",.,Never married,No,Under 12 Lak

In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Referred by - share row# I name I city (this field is not mandatory - those who are referred need to fill this),Video Introduction
2,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Health concious due to health conditions in fa...,I give consent,,,,,,,,
3,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,I like playing sports,I give consent,,,,,,,,
4,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,No,I give consent,,,,,,,,
5,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,I take time to open up and quite curious,I give consent,,,,,,,,
6,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,I believe honest communication and mutual resp...,I give consent,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,Male,36,Pune,,"A life with a few friends bt real ones, a job ...",Grew up in a middle class strict family to rea...,"Was married earlier, legally divorced",No,Between 18-20 Lakhs,I am a professional Chef,...,"I love cars, bikes and dogs \nWould help an an...",I give consent,Hindu,Yes,I am not sure,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
437,Male,38,Delhi,,"Healthy, loving family and good job",Awesome,"Was married earlier, legally divorced",No,Between 40-50 Lakhs,IT professional,...,No,I give consent,Hindu,Yes,It will not work for me,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
438,Female,31,Bangalore,,Having someone cheer for you when you follow y...,Growing up years were decent. No major complai...,Never married,No,Between 40-50 Lakhs,Software engineer,...,"Into Fitness, beaches and baby elephants :P Re...",I give consent,Other,I am not sure,It will not work for me,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
439,Male,33,Bangalore,,"Supportive, Caring, Patience.",.,Never married,No,Under 12 Lak

In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
436,Male,36,Pune,"Was married earlier, legally divorced"
437,Male,38,Delhi,"Was married earlier, legally divorced"
438,Female,31,Bangalore,Never married
439,Male,33,Bangalore,Never married


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Legally Separated,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,"Was married earlier, legally divorced",Widow/ Widower
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0
5,0,0,0,0,1,0,0
6,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
436,0,0,0,0,0,1,0
437,0,0,0,0,0,1,0
438,0,1,0,0,0,0,0
439,0,1,0,0,0,0,0


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
436,Male,36,Pune,"Was married earlier, legally divorced"
437,Male,38,Delhi,"Was married earlier, legally divorced"
438,Female,31,Bangalore,Never married
439,Male,33,Bangalore,Never married


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
436,Male,36,Pune,"Was married earlier, legally divorced"
437,Male,38,Delhi,"Was married earlier, legally divorced"
438,Female,31,Bangalore,Never married
439,Male,33,Bangalore,Never married


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(439, 155)


In [17]:
k =280
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=280)

In [18]:
row =210
selectedRow = refDataset.iloc[row]

In [19]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 401 109 252 409 424 396 316  11 108 372 272 266 165  45 342
  313  25 331 320  41 330 278 385 343 326 383 332 247 245 290 261 156  48
  351  24 311  81 229 386  26 309 358 425 139 181 256 123 354 363 333 288
  118  97  51  30 122  54 390 295 128 110 376 262 412 136  84  86 349 414
   19 381 260 231  75  55 259 325 274 336 310 114 307 365 393 183  42 375
  355 265 287 308 415  52 200 353 174  27   1 378 100  32 404 105  73  50
   18   6 185 327  29 228 258 344 251  65 111  12 301 178 282 221 246 373
  291 388 121 435  66 296  96 420  77 345 406 166 201  83 348 244  56 196
  237  36 169 428 380 135 411 106 209 340 328 160 143 241 432 194  89  91
  163  71 208 219 171 357 255 335 421 107 379 352 321 230  76 184 314 438
  112 238 271 104 264  21 211 403 173 422 263  53  35 382 408 119 214 249
  413 205 147 142 141 242  70  94 191  79 391 137  62 180 315  37  59 426
    9 405 188 152 346 101 257 297 429 179 324  61  23  57 276 368 117 293
    8 361 131  60 212 126  43  13  88 

In [20]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 43 'Delhi' 'Never married']


['Male' 47 'New Delhi ' 'Never married']


['Male' 55 'Dubai ' 'Never married']


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 50 'Delhi ' 'Separated, not legally divorced']


['Male' 49 'Pune' 'Never married']


['Male' 49 'Bangalore, Karnataka, India' 'Never married']


['Male' 49 'Pune' 'Never married']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 45 'Mumbai ' 'Never married']


['Male' 37 'Delhi' 'Never married']


['Male' 44 'Delhi' 'Widow/ Widower']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 36 'Delhi' 'Never married']


['Male' 36 'Delhi ' 'Never married']


['Male' 42 'Bangalore ' 'Never married']


['Male' 42 'Mumbai' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 55 'Gurgaon' 'Separated, not legally divorced']


['Male' 45 'New Delhi ' 'Was married earlier

In [21]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [22]:
print(preprocessData(refDataset))

[[ 1.         -0.72336243  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.22063508  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.89093821  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.         -1.05851399  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.72336243  0.         ...  0.          0.
   0.        ]
 [ 0.          0.61724383  0.         ...  0.          0.
   0.        ]]


In [23]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']